# Tutorial - Evaluate DNBs additional Rules (QRS)

This notebook is a tutorial for the evaluation of DNBs additional Rules for the quarterly Solvency II reports for solo entities.

## Import packages

In [1]:
from arelle import ModelManager, Cntlr, ModelFormulaObject, ModelXbrl, ViewFileFormulae, XbrlConst, ViewFileRenderedGrid
from arelle import RenderingEvaluator

import pandas as pd  # dataframes
import numpy as np  # mathematical functions, arrays and matrices
from os.path import join, isfile  # some os dependent functionality
import re  # regular expressions
from src import Evaluator  # conversion from 'rules' to expressions for the data-patterns packages
import data_patterns  # evaluation of patterns
from pprint import pprint  # pretty print
import logging

## General parameters

In [2]:
# path to the source file with the additional rules
RULES_PATH = join('..', 'solvency2-rules')  
# path to the source data
INSTANCES_DATA_PATH = join('..', 'data', 'instances', 'qrs_240_instance')
# path to the results
RESULTS_PATH = join('..', 'results') 
# input parameters
PARAMETERS = {'decimal': 0}  
# currently only 'decimal' is availabl,e which specifies tolerance during evaluation of patterns.
# decimal: 0 means tolerance = abs(1.5e-0) (= 1.5)

In [3]:
# We log to rules.log in the data/instances path
logging.basicConfig(filename = join(INSTANCES_DATA_PATH, 'rules.log'),level = logging.INFO, 
                    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s')

## Read file with all possible datapoints

We use a simplified taxonomy with all possible datapoints, located in the data/datapoints directory

In [4]:
DATAPOINTS_PATH = join('..', 'data', 'datapoints')  # path to file
FILENAME_DATAPOINTS = 'QRS.csv'  # filename
df_datapoints = pd.read_csv(join(DATAPOINTS_PATH, FILENAME_DATAPOINTS), sep=";")  # load file to dataframe
df_datapoints.head()

,tabelcode,datapunt
0,S.01.01.02.01,R0010C0010
1,S.01.01.02.01,R0030C0010
2,S.01.01.02.01,R0110C0010
3,S.01.01.02.01,R0140C0010
4,S.01.01.02.01,R0150C0010


In [5]:
# Add columns with only R(ow) and C(olumn) reference:
df_datapoints['rij'] = df_datapoints['datapunt'].apply(lambda x: "" if re.search("R\d\d\d\d", x) is None else re.search("R\d\d\d\d", x)[0])
df_datapoints['kolom'] = df_datapoints['datapunt'].apply(lambda x: "" if re.search("C\d\d\d\d", x) is None else re.search("C\d\d\d\d", x)[0])
df_datapoints.head()

,tabelcode,datapunt,rij,kolom
0,S.01.01.02.01,R0010C0010,R0010,C0010
1,S.01.01.02.01,R0030C0010,R0030,C0010
2,S.01.01.02.01,R0110C0010,R0110,C0010
3,S.01.01.02.01,R0140C0010,R0140,C0010
4,S.01.01.02.01,R0150C0010,R0150,C0010


## Read input data

We distinguish 2 types of tables: with a closed axis, and with an open axis.

An example of a table with an open axis is the list of assets: an entity reports several 'rows of data' in the relevant table. An example of a closed axis is the balance sheet: an entity reports only 1 balance sheet per period.

### Read tables from source path

We combine all tables with closed axes into one DataFrame. This DataFrame is then used for all validation rules for closed axes tables. 

Tables with an open axis are put in a dictionary of DataFrames. We perform validation rules per tables with an open axis.

In [6]:
tables_complete_set = df_datapoints.tabelcode.sort_values().unique().tolist()  # list of all QRS tables
tables = [table for table in tables_complete_set
          if isfile(join(INSTANCES_DATA_PATH, table + '.pickle'))]  # QRS tables found in the input folder
tables_closed_axis = []  # for listing all input tables with closed axis
tables_open_axis = []  # for listing all input tables with open axis
df_closed_axis = pd.DataFrame()  # one dataframe with all data from closed axis tables
dict_open_axis = {}  # dictionary with all open axis tables

for table in tables:
    df = pd.read_pickle(join(INSTANCES_DATA_PATH, table + '.pickle'))  # read dataframe
    
    if df.index.nlevels > 2:  # if more than 2 indexes (entity, period) --> open axis
        # Identify which columns within an open axis table make a 'table row' unique (index-columns):
        index_columns_open_axis = [col for col in list(df.index.names) if col not in ['entity','period']]
        
        # Duplicate index-columns to data columns:
        df.reset_index(level=index_columns_open_axis, inplace=True)
        for i in range(len(index_columns_open_axis)):
            df['index_col_' + str(i)] = df[index_columns_open_axis[i]].astype(str)
            df.set_index(['index_col_' + str(i)], append=True, inplace=True)
        
        # Add to relevant list/dict
        tables_open_axis.append(table)
        dict_open_axis[table] = df
    else:  # closed axis
        tables_closed_axis.append(table)  # add to relevant list
        # Add table to dataframe with all data from closed axis tables
        if len(df_closed_axis) == 0:  # no data yet --> copy dataframe
            df_closed_axis = df.copy()
        else:  # join to existing dataframe
            df_closed_axis = df_closed_axis.join(df)

print('Closed-axis tables:')
pprint(tables_closed_axis)
print()
print('Open-axis tables:')
pprint(tables_open_axis)

Closed-axis tables:
['S.01.01.02.01',
 'S.01.02.01.01',
 'S.02.01.02.01',
 'S.05.01.02.01',
 'S.05.01.02.02',
 'S.12.01.02.01',
 'S.17.01.02.01',
 'S.23.01.01.01',
 'S.23.01.01.02',
 'S.28.01.01.01',
 'S.28.01.01.02',
 'S.28.01.01.03',
 'S.28.01.01.04',
 'S.28.01.01.05',
 'S.28.02.01.01',
 'S.28.02.01.02',
 'S.28.02.01.03',
 'S.28.02.01.04',
 'S.28.02.01.05',
 'S.28.02.01.06']

Open-axis tables:
['S.06.02.01.01',
 'S.06.02.01.02',
 'S.06.03.01.01',
 'S.08.01.01.01',
 'S.08.01.01.02',
 'S.08.02.01.01',
 'S.08.02.01.02']


### Add not reported datapoints as 0's to the dataframes

Here we perform some necessary data cleaning.

In [7]:
# List with all possible datapoints:
all_datapoints = [x.replace(',,',',') for x in list(df_datapoints['tabelcode'] + ',' + df_datapoints['rij'] + ',' + df_datapoints['kolom'])]
# List with all possible datapoints for closed axis tables:
all_datapoints_closed = [x for x in all_datapoints if x[:13] in tables_closed_axis]
# List with all possible datapoints for open axis tables:
all_datapoints_open = [x for x in all_datapoints if x[:13] in tables_open_axis]

# Add not reported datapoints to the dataframe with data from closed axis tables
for column in [column for column in all_datapoints_closed if column not in list(df_closed_axis.columns)]:
    df_closed_axis[column] = np.NaN
df_closed_axis.fillna(0, inplace = True)

# Add not reported datapoints to the dataframes with data from open axis tables
for table in [table for table in dict_open_axis.keys()]:
    all_datapoints_table = [x for x in all_datapoints_open if x[:13] == table]
    for column in [column for column in all_datapoints_table if column not in list(dict_open_axis[table].columns)]:
        dict_open_axis[table][column] = np.NaN
    dict_open_axis[table].fillna(0, inplace = True)

### Compare reported tables with the complete set

In [8]:
diff = list(np.setdiff1d(tables_complete_set, tables))
if len(diff) == 0:
    print('Found files for all possible tables')
else:
    print('No file found for the following tables:', diff)

Found files for all possible tables


## Read DNBs Additional Validation Rules

DNBs additional validation rules are currently published as an Excel file on the DNB statistics website. We included the Excel file here in the project.

Here we read the Excel and perform some data cleaning.

In [9]:
FILENAME_RULES = '2020-01-22 Set aanvullende controleregels Solvency II_tcm46-387021.xlsx'

def read_rules(entrypoint):  # function for loading the addtional rules to a dataframe, with filter on entrypoint (i.e. QRS)
    df = pd.read_excel(join(RULES_PATH, FILENAME_RULES), header = 1)
    df = df[(df['Standaard'] == 'SOLVENCY') | (df['Standaard'] == entrypoint)]
    df.drop_duplicates(inplace=True) #remove double lines
    df.fillna("", inplace = True)
    df = df.set_index('ControleRegelCode')
    return df

In [10]:
df_rules = read_rules('QRS')
df_rules.head()

,ControleRegelType,HoofdTabel,Rijen,Kolommen,Formule,Meldtekst,Standaard,TolerantieType
ControleRegelCode,,,,,,,,
S.01.01_112,C,S.01.01.02.01,,,"IF {S.01.01.02.01,r0580,c0010} = ""REPORTED"" TH...",Een van de MCR formulieren is van toepassing,SOLVENCY,XBRL
S.01.01_113,C,S.01.01.02.01,,,"{S.01.01.02.01,r0590,c0010} <> ""REPORTED ""","S.28.01.01 (R0590,c0010) Minimum Capital Requ...",SOLVENCY,XBRL
S.01.02_102,C,S.01.02.01.01,,,"{S.01.02.01.01,r0050,c0010} = ""NETHERLANDS""","U heeft bij vraag R0050,c0010 op formulier S.0...",SOLVENCY,XBRL
S.01.02_103,C,S.01.02.04.01,,,"{S.01.02.04.01,r0050,c0010} = ""NETHERLANDS""","U heeft bij vraag R0050,c0010 op formulier S.0...",SOLVENCY,XBRL
S.01.02_104,C,S.01.02.01.01,,,"{S.01.02.01.01,r0070,c0010} = ""DUTCH"" OR {S.01...","U heeft bij vraag R0050,C0010 op formulier S.0...",SOLVENCY,XBRL


## Evaluate rules for tables with a closed axis

The evaluator is a piece of Python code, which takes the Additional Validation Rules as input, and transforms it to expressions that can be interpreted by the data_patterns package.

The data-patterns package is also called from within the evaluator, to evaluate the results.

Here we evaluate the validation rules for tables with closed axes (that are put into a single dataframe).

In [11]:
evaluator = Evaluator(df_closed_axis, df_rules, df_datapoints, PARAMETERS)

In [12]:
evaluator.df_patterns.head()  # resulting patterns for closed axis tables

,pattern_id,cluster,pattern_def,support,exceptions,confidence,pattern status,encodings,pandas co,pandas ex,xbrl co,xbrl ex,Error message
index,,,,,,,,,,,,,
0,S.01.01_112,0,"IF {""S.01.01.02.01,R0580,C0010""} = ""REPORTED"" ...",0,0,0.0,DNB,{},"df[(df['S.01.01.02.01,R0580,C0010']=='REPORTED...","df[(df['S.01.01.02.01,R0580,C0010']=='REPORTED...",,,
1,S.01.01_113,0,"{""S.01.01.02.01,R0590,C0010""} != ""REPORTED """,1,0,1.0,DNB,{},"df[(df['S.01.01.02.01,R0590,C0010']!='REPORTED...","df[~(df['S.01.01.02.01,R0590,C0010']!='REPORTE...",,,
2,S.01.02_102,0,"{""S.01.02.01.01,R0050,C0010""} = ""NETHERLANDS""",0,1,0.0,DNB,{},"df[(df['S.01.02.01.01,R0050,C0010']=='NETHERLA...","df[~(df['S.01.02.01.01,R0050,C0010']=='NETHERL...",,,
3,S.01.02_104,0,"{""S.01.02.01.01,R0070,C0010""} = ""DUTCH"" | {""S....",0,1,0.0,DNB,{},"df[((df['S.01.02.01.01,R0070,C0010']=='DUTCH')...","df[~((df['S.01.02.01.01,R0070,C0010']=='DUTCH'...",,,
4,S.01.02_110,0,"{""S.01.02.01.01,R0100,C0010""} = ""REGULAR REPOR...",0,1,0.0,DNB,{},"df[(df['S.01.02.01.01,R0100,C0010']=='REGULAR ...","df[~(df['S.01.02.01.01,R0100,C0010']=='REGULAR...",,,


In [13]:
evaluator.df_results.head()  # results for closed axis tables

result_type   pattern_id  cluster  support  \
0LFF1WMNTWG5PTIYYI38 2019-12-31         True  S.01.01_113        0        1   
                     2019-12-31         True  S.02.01_106        0        1   
                     2019-12-31         True  S.02.01_106        0        1   
                     2019-12-31         True  S.05.01_113        0        1   
                     2019-12-31         True  S.05.01_113        0        1   

                                 exceptions  confidence  \
0LFF1WMNTWG5PTIYYI38 2019-12-31           0         1.0   
                     2019-12-31           0         1.0   
                     2019-12-31           0         1.0   
                     2019-12-31           0         1.0   
                     2019-12-31           0         1.0   

                                                                       pattern_def  \
0LFF1WMNTWG5PTIYYI38 2019-12-31       {"S.01.01.02.01,R0590,C0010"} != "REPORTED "   
                     2019-12-31                {"S.02.01.02.01,R0040,C0010"} >= 0    
                     2019-12-31                {"S.02.01.02.01,R0780,C0010"} >= 0    
                     2019-12-31  IF {"S.05.01.02.01,R0110,C0010"} != 0|{"S.05.0...   
                     2019-12-31  IF {"S.05.01.02.01,R0110,C0020"} != 0|{"S.05.0...   

                                                                          P values  \
0LFF1WMNTWG5PTIYYI38 2019-12-31  Not reported as only life or only non-life ins...   
                     2019-12-31                                        8.16314e+08   
                     2019-12-31                                        1.31643e+08   
                     2019-12-31                       [152945460.23, 320026354.89]   
                     2019-12-31                         [230134597.93, 46053138.3]   

                                    Q values  
0LFF1WMNTWG5PTIYYI38 2019-12-31               
                     2019-12-31               
                     2019-12-31               
                     2019-12-31  5.03392e+08  
                     2019-12-31   2.5025e+08

## Evaluate rules for tables with an open axis

Then we perform the relevant validation rules for each table with an open axis separately.

### Check if there are rules for tables with an open axis

In [14]:
tables_with_rules = set([a for b in evaluator.df_rules.templates.tolist() for a in b])
tables_with_open_axis_and_rules = tables_with_rules & set(tables_open_axis)
print(tables_with_open_axis_and_rules)

{'S.06.03.01.01'}


### Evaluate rules for tables with an open axis

In [15]:
output_open_axis = {}  # dictionary with input and results per table
for table in tables_with_open_axis_and_rules:  # loop through open axis tables for which a rule has been defined
    info = {}
    info['data'] = dict_open_axis[table]  # add data to dictionary
    rule_indexes = evaluator.df_rules[evaluator.df_rules['templates'].
                                      apply(lambda x: any([table in x]))].index  # identify rules for current table
    info['rules'] = df_rules.loc[rule_indexes]  # add rules to dictionary
    evaluator_open_axis = Evaluator(info['data'], info['rules'], df_datapoints, PARAMETERS)  # call evaluator
    info['patterns'] = evaluator_open_axis.df_patterns  # add resulting patterns to dictionary
    info['results'] = evaluator_open_axis.df_results  # add results to dictionary
    output_open_axis[table] = info

In [16]:
if len(tables_with_open_axis_and_rules) > 0:  # print rules if there are rules for tables with an open axis (for first table)
    display(output_open_axis[list(tables_with_open_axis_and_rules)[0]]['rules'])

,ControleRegelType,HoofdTabel,Rijen,Kolommen,Formule,Meldtekst,Standaard,TolerantieType,datapoints,templates,Formule_input,error,n_patterns
ControleRegelCode,,,,,,,,,,,,,
S.06.03_100,C,S.06.03.01.01,(All),,"IF {S.06.03.01.01,c0010}<>"" "" THEN {S.06.03.01...",Underlying asset categoty is niet vermeld,SOLVENCY,XBRL,"[S.06.03.01.01,C0010, S.06.03.01.01,C0030]",[S.06.03.01.01],"IF {""S.06.03.01.01,C0010""}!="" "" THEN {""S.06.03...",,1


In [17]:
if len(tables_with_open_axis_and_rules) > 0:  # print patterns if there are rules for tables with an open axis (for first table)
    display(output_open_axis[list(tables_with_open_axis_and_rules)[0]]['patterns'])

,pattern_id,cluster,pattern_def,support,exceptions,confidence,pattern status,encodings,pandas co,pandas ex,xbrl co,xbrl ex,Error message
index,,,,,,,,,,,,,
0,S.06.03_100,0,"IF {""S.06.03.01.01,C0010""}!="" "" THEN {""S.06.03...",1,0,1.0,DNB,{},"df[(df['S.06.03.01.01,C0010']!=' ') & (df['S.0...","df[(df['S.06.03.01.01,C0010']!=' ') & ~(df['S....",,,


In [18]:
if len(tables_with_open_axis_and_rules) > 0:  # print results if there are rules for tables with an open axis (for first table)
    display(output_open_axis[list(tables_with_open_axis_and_rules)[0]]['results'].head())

,,,,result_type,pattern_id,cluster,support,exceptions,confidence,pattern_def,P values,Q values
0LFF1WMNTWG5PTIYYI38,2019-12-31,1,ISIN/XV2000282645,True,S.06.03_100,0,1,0,1.0,"IF {""S.06.03.01.01,C0010""}!="" "" THEN {""S.06.03...",ISIN/XV2000282645,Collective investments undertakings


## Combine and export results for closed and open axis tables

Now we have the results from tables with closed and open axes and we can combine the results.

In [19]:
# Function to transform results for open-axis tables, so it can be appended to results for closed-axis tables
# The 'extra' index columns are converted to data columns
def transform_results_open_axis(df):
    if df.index.nlevels > 2:
        reset_index_levels = list(range(2, df.index.nlevels))
        df = df.reset_index(level=reset_index_levels)
        rename_columns={}
        for x in reset_index_levels:
            rename_columns['level_' + str(x)] = 'id_column_' + str(x - 1)
        df.rename(columns=rename_columns, inplace=True)
    return df

In [20]:
df_all_patterns = evaluator.df_patterns  # patterns for closed axis tables 
df_all_results = evaluator.df_results  # results for closed axis tables
for table in output_open_axis:  # for all open axis tables with rules -> append and sort patterns and results
    df_all_patterns = output_open_axis[table]['patterns'].append(df_all_patterns).sort_values(by=['pattern_id']).reset_index(drop=True)
    df_all_results = transform_results_open_axis(output_open_axis[table]['results']).append(df_all_results, sort=False).sort_values(by=['pattern_id']).sort_index()

## Save results

The dataframe df_all_results contains all output of the evaluation of the validation rules. 

In [21]:
df_all_patterns.to_excel(join(RESULTS_PATH, "patterns.xlsx"))  # export patterns to excel file in results folder

In [22]:
# To save all results use df_all_results
# To save all exceptions use df_all_results['result_type']==False 
# To save all confirmations use df_all_results['result_type']==True

# Here we save only the exceptions to the validation rules
df_results = df_all_results[df_all_results['result_type']==False]
df_results.to_excel(join(RESULTS_PATH, "results.xlsx"))  # export results to excel file in results folder

In [23]:
df_all_patterns.head()

,pattern_id,cluster,pattern_def,support,exceptions,confidence,pattern status,encodings,pandas co,pandas ex,xbrl co,xbrl ex,Error message
0,S.01.01_112,0,"IF {""S.01.01.02.01,R0580,C0010""} = ""REPORTED"" ...",0,0,0.0,DNB,{},"df[(df['S.01.01.02.01,R0580,C0010']=='REPORTED...","df[(df['S.01.01.02.01,R0580,C0010']=='REPORTED...",,,
1,S.01.01_113,0,"{""S.01.01.02.01,R0590,C0010""} != ""REPORTED """,1,0,1.0,DNB,{},"df[(df['S.01.01.02.01,R0590,C0010']!='REPORTED...","df[~(df['S.01.01.02.01,R0590,C0010']!='REPORTE...",,,
2,S.01.02_102,0,"{""S.01.02.01.01,R0050,C0010""} = ""NETHERLANDS""",0,1,0.0,DNB,{},"df[(df['S.01.02.01.01,R0050,C0010']=='NETHERLA...","df[~(df['S.01.02.01.01,R0050,C0010']=='NETHERL...",,,
3,S.01.02_104,0,"{""S.01.02.01.01,R0070,C0010""} = ""DUTCH"" | {""S....",0,1,0.0,DNB,{},"df[((df['S.01.02.01.01,R0070,C0010']=='DUTCH')...","df[~((df['S.01.02.01.01,R0070,C0010']=='DUTCH'...",,,
4,S.01.02_110,0,"{""S.01.02.01.01,R0100,C0010""} = ""REGULAR REPOR...",0,1,0.0,DNB,{},"df[(df['S.01.02.01.01,R0100,C0010']=='REGULAR ...","df[~(df['S.01.02.01.01,R0100,C0010']=='REGULAR...",,,


In [24]:
df_results.head()

id_column_1 id_column_2  result_type  \
0LFF1WMNTWG5PTIYYI38 2019-12-31         NaN         NaN        False   
                     2019-12-31         NaN         NaN        False   
                     2019-12-31         NaN         NaN        False   
                     2019-12-31         NaN         NaN        False   
                     2019-12-31         NaN         NaN        False   

                                  pattern_id  cluster  support  exceptions  \
0LFF1WMNTWG5PTIYYI38 2019-12-31  S.01.02_102        0        0           1   
                     2019-12-31  S.01.02_104        0        0           1   
                     2019-12-31  S.01.02_110        0        0           1   
                     2019-12-31  S.02.01_116        0        0           1   
                     2019-12-31  S.17.01_119        0        0           1   

                                 confidence  \
0LFF1WMNTWG5PTIYYI38 2019-12-31         0.0   
                     2019-12-31         0.0   
                     2019-12-31         0.0   
                     2019-12-31         0.0   
                     2019-12-31         0.0   

                                                                       pattern_def  \
0LFF1WMNTWG5PTIYYI38 2019-12-31      {"S.01.02.01.01,R0050,C0010"} = "NETHERLANDS"   
                     2019-12-31  {"S.01.02.01.01,R0070,C0010"} = "DUTCH" | {"S....   
                     2019-12-31  {"S.01.02.01.01,R0100,C0010"} = "REGULAR REPOR...   
                     2019-12-31  {"S.02.01.02.01,R0610,C0010"} + {"S.02.01.02.0...   
                     2019-12-31  {"S.17.01.02.01,R0330,C0170"} = {"S.17.01.02.0...   

                                                                          P values  \
0LFF1WMNTWG5PTIYYI38 2019-12-31                                         MARTINIQUE   
                     2019-12-31                                     Serbo-Croatian   
                     2019-12-31  Re-submission of S.30 templates in accordance ...   
                     2019-12-31         [683920894.64, 641163639.33, 370406105.97]   
                     2019-12-31                                        8.58816e+08   

                                                                          Q values  
0LFF1WMNTWG5PTIYYI38 2019-12-31                                                     
                     2019-12-31                                                     
                     2019-12-31                                                     
                     2019-12-31  [933717078.28, 55071374.05, 494295836.93, 4419...  
                     2019-12-31            [466837334.79, 5827857.53, 364768433.0]

In [25]:
# Get the pandas code from the first pattern and evaluate it
s = df_all_patterns.loc[2, 'pandas ex'].replace('df', 'df_closed_axis')
print(s)
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # print whole dataframe
    display(eval(s).T)

df_closed_axis[~(df_closed_axis['S.01.02.01.01,R0050,C0010']=='NETHERLANDS')]


entity,0LFF1WMNTWG5PTIYYI38
period,2019-12-31
"S.01.01.02.01,R0010,C0010",Reported
"S.01.01.02.01,R0030,C0010",Exempted under Article 35 (6) to (8)
"S.01.01.02.01,R0110,C0010",Exempted under Article 35 (6) to (8)
"S.01.01.02.01,R0140,C0010",Not due annually as reported for Quarter 4
"S.01.01.02.01,R0150,C0010",Reported
"S.01.01.02.01,R0170,C0010",Not reported other reason
"S.01.01.02.01,R0180,C0010",Not reported o/a no derivative transactions
"S.01.01.02.01,R0220,C0010",Reported
"S.01.01.02.01,R0290,C0010",Exempted under Article 35 (6) to (8)
